In [5]:
import numpy as np
import pandas as pd

df = pd.read_excel('talkdata.xlsx')
df.head(3)

,chatID,personType,personID,createdAt,plainText
0,Jac17549d6f7v,manager,T8758793187x,2021-01-04 10:32:35,안녕하세요 KULTIPS에 가이드 문서를 제공하겠습니다.
1,Ub173640ed31u,manager,W6523714365l,2021-01-04 09:07:18,"기본적으로 영어로 코스가 생성된 경우, [Properties]에서 언어를 한국어로 ..."
2,Ub173640ed31u,user,N4714897d8f7t,2021-01-04 11:39:41,네.. 저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요. ...


In [6]:
text = df['plainText'].tolist()
print(len(text))

65373


In [18]:
user = df.query('personType == "user"')
user

,chatID,personType,personID,createdAt,plainText
2,Ub173640ed31u,user,N4714897d8f7t,2021-01-04 11:39:41,네.. 저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요. ...
6,Rf912748ed75b,user,G09261264c9ar,2021-01-04 09:08:32,보낸 후 따로 해결했습니다
7,V6744149ed76k,user,Bcc18665a2c4u,2021-01-04 14:22:57,감사합니다!
9,V6744149ed76k,user,Bcc18665a2c4u,2021-01-04 14:42:51,아니요 감사합니다
12,Ace13424d164l,user,Hb6754774b33z,2021-01-05 10:26:53,"안녕하세요,혹시 교수님이 특정학생이 컨텐츠에 접근했는지 여부를 확인할 수 있는 방법..."
...,...,...,...,...,...
65364,F3143068ceafd,user,Fc758257f2ddo,2021-12-31 16:35:15,Cheers
65367,F3143068ceafd,user,Fc758257f2ddo,2021-12-31 16:36:42,Happy New Year to you too!
65368,K0510513cebcc,user,H09257733c6dm,2021-12-31 17:17:18,"안녕하십니까, 장애 인식개선 교육에 관해 여쭈어볼 것이 있어 연락 드렸습니다. 장애..."
65371,Vbb38262cef7i,user,Q15266634a8br,2021-12-31 21:30:39,"인권과 성평등 교육 17시 이후에도 이수할 수 있던데, 31일 안으로만 이수하면 되..."


In [31]:
user_text = user['plainText'].tolist()
print(len(user_text))

29844


In [27]:
import re
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter

In [35]:
user = user.reset_index(drop=True)
user.head(3)

,chatID,personType,personID,createdAt,plainText
0,Ub173640ed31u,user,N4714897d8f7t,2021-01-04 11:39:41,네.. 저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요. ...
1,Rf912748ed75b,user,G09261264c9ar,2021-01-04 09:08:32,보낸 후 따로 해결했습니다
2,V6744149ed76k,user,Bcc18665a2c4u,2021-01-04 14:22:57,감사합니다!


In [29]:
# 한글 표현만 남기기
def extract_word(user_text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', user_text)
    return result

In [37]:
print("Before : ",user['plainText'][0])
print("After : ",extract_word(user['plainText'][0]))

Before :  네.. 저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요. 설정을 변경할 수 없다면 방법이 없겠네요. 답변주셔서 감사합니다.
After :  네   저도 불편하지만 학생들도 메뉴를 한눈에 알아보기 어려워서 어려움이 많아요  설정을 변경할 수 없다면 방법이 없겠네요  답변주셔서 감사합니다 


In [57]:
user['plainText'] = user['plainText'].apply(lambda x: extract_word(x))

In [58]:
# 형태소 분석
okt = Okt()
words = " ".join(user['plainText'].tolist())
words = okt.morphs(words,stem=True)

len(words)

317329

In [59]:
remove_one_word = [x for x in words if len(x)>1]
len(remove_one_word)

222951

In [60]:
from collections import Counter
frequent = Counter(remove_one_word).most_common()

In [61]:
frequent

[('하다', 15896),
 ('있다', 5873),
 ('되다', 4828),
 ('감사하다', 4561),
 ('으로', 2714),
 ('수업', 2486),
 ('에서', 2465),
 ('안녕하다', 2440),
 ('않다', 2368),
 ('블랙보드', 2355),
 ('강의', 2354),
 ('학생', 2300),
 ('확인', 2274),
 ('이다', 2174),
 ('없다', 1994),
 ('교수', 1960),
 ('드리다', 1933),
 ('어떻다', 1843),
 ('뜨다', 1784),
 ('출석', 1667),
 ('알다', 1614),
 ('보다', 1518),
 ('되어다', 1514),
 ('문제', 1397),
 ('문의', 1388),
 ('안되다', 1354),
 ('시험', 1348),
 ('혹시', 1307),
 ('같다', 1286),
 ('방법', 1239),
 ('아니다', 1145),
 ('다시', 1070),
 ('접속', 1036),
 ('그렇다', 1032),
 ('영상', 1021),
 ('설정', 1006),
 ('들어가다', 978),
 ('조교', 950),
 ('하고', 935),
 ('제출', 933),
 ('과제', 902),
 ('녹화', 888),
 ('나오다', 872),
 ('등록', 858),
 ('해결', 800),
 ('화면', 790),
 ('다른', 786),
 ('로그인', 781),
 ('시간', 769),
 ('코스', 754),
 ('해보다', 742),
 ('가능하다', 732),
 ('받다', 705),
 ('콜라보', 702),
 ('맞다', 697),
 ('들다', 695),
 ('락다운', 677),
 ('사용', 671),
 ('경우', 670),
 ('수강', 657),
 ('지금', 655),
 ('누르다', 651),
 ('파일', 649),
 ('보이다', 646),
 ('과목', 631),
 ('올리다', 612),
 ('브라우저', 604),

In [62]:
with open('stopwords.txt','r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
len(remove_stopwords)

144930

In [63]:
Counter(remove_stopwords).most_common()

[('감사하다', 4561),
 ('수업', 2486),
 ('안녕하다', 2440),
 ('블랙보드', 2355),
 ('강의', 2354),
 ('학생', 2300),
 ('확인', 2274),
 ('교수', 1960),
 ('드리다', 1933),
 ('뜨다', 1784),
 ('출석', 1667),
 ('문제', 1397),
 ('문의', 1388),
 ('시험', 1348),
 ('방법', 1239),
 ('접속', 1036),
 ('영상', 1021),
 ('설정', 1006),
 ('들어가다', 978),
 ('조교', 950),
 ('제출', 933),
 ('과제', 902),
 ('녹화', 888),
 ('등록', 858),
 ('해결', 800),
 ('화면', 790),
 ('로그인', 781),
 ('코스', 754),
 ('해보다', 742),
 ('가능하다', 732),
 ('콜라보', 702),
 ('맞다', 697),
 ('락다운', 677),
 ('사용', 671),
 ('경우', 670),
 ('수강', 657),
 ('누르다', 651),
 ('파일', 649),
 ('과목', 631),
 ('올리다', 612),
 ('브라우저', 604),
 ('레이', 603),
 ('하나요', 590),
 ('자다', 568),
 ('께서', 552),
 ('진행', 548),
 ('답변', 547),
 ('보내다', 546),
 ('관련', 528),
 ('좋다', 522),
 ('자료', 522),
 ('해주다', 519),
 ('체크', 513),
 ('메뉴', 511),
 ('학기', 506),
 ('기록', 478),
 ('오류', 463),
 ('궁금하다', 446),
 ('업로드', 446),
 ('삭제', 440),
 ('해당', 437),
 ('메일', 434),
 ('연락', 432),
 ('링크', 423),
 ('교육', 420),
 ('건가', 407),
 ('여쭈다', 405),
 ('질문', 398),
 ('하

In [64]:
with open('stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
len(remove_stopwords)

144930

In [65]:
Counter(remove_stopwords).most_common()

[('감사하다', 4561),
 ('수업', 2486),
 ('안녕하다', 2440),
 ('블랙보드', 2355),
 ('강의', 2354),
 ('학생', 2300),
 ('확인', 2274),
 ('교수', 1960),
 ('드리다', 1933),
 ('뜨다', 1784),
 ('출석', 1667),
 ('문제', 1397),
 ('문의', 1388),
 ('시험', 1348),
 ('방법', 1239),
 ('접속', 1036),
 ('영상', 1021),
 ('설정', 1006),
 ('들어가다', 978),
 ('조교', 950),
 ('제출', 933),
 ('과제', 902),
 ('녹화', 888),
 ('등록', 858),
 ('해결', 800),
 ('화면', 790),
 ('로그인', 781),
 ('코스', 754),
 ('해보다', 742),
 ('가능하다', 732),
 ('콜라보', 702),
 ('맞다', 697),
 ('락다운', 677),
 ('사용', 671),
 ('경우', 670),
 ('수강', 657),
 ('누르다', 651),
 ('파일', 649),
 ('과목', 631),
 ('올리다', 612),
 ('브라우저', 604),
 ('레이', 603),
 ('하나요', 590),
 ('자다', 568),
 ('께서', 552),
 ('진행', 548),
 ('답변', 547),
 ('보내다', 546),
 ('관련', 528),
 ('좋다', 522),
 ('자료', 522),
 ('해주다', 519),
 ('체크', 513),
 ('메뉴', 511),
 ('학기', 506),
 ('기록', 478),
 ('오류', 463),
 ('궁금하다', 446),
 ('업로드', 446),
 ('삭제', 440),
 ('해당', 437),
 ('메일', 434),
 ('연락', 432),
 ('링크', 423),
 ('교육', 420),
 ('건가', 407),
 ('여쭈다', 405),
 ('질문', 398),
 ('하

In [66]:
# 최소 횟수 이하 단어 제거
for item, count in Counter(remove_stopwords).most_common():
    if count==1:
        print(item)

수처리
정당하다
재체점
사건
김재일
분제
요점
대별
지적재산권
폴더탭
암벽등반
안고
이루
야호
심의
지라
튕기거
무음
낼때
다람쥐
캐쉬
선형대수학
만으로도
다봣는데
하셧
셔서
증원
홍보
포스터
유혜정
말기
폭력
이지영
개방
반디캠
균등하다
배분
표지
빠빠이
한국인
우싱
뮬리
만화
이차전지
금주
고지서
장기간
디다
부임
정연
안이
나르다
박진성
부합
지적재산
콜렙
제품디자인
성업
발습
대학생
사이버
궁금힙니
어딨다
일산
물릿
음영
동이
미학
송희경입니
벌어지다
상강
학과목
자료구조
지성
영국
중임
토픽
우선권
거절
쪼개다
이민지
쿠타
국어생활
좌측탭
수험
후속
등재
이문
강위
신호처리
김태곤
형사법
힌대
콜로
이벤트
동하
못내
될떄
삼국유사
도데
쭈르륵
유니코드
이랑은
왓는데
드레스
지란
콜레보
젠더
볼링
목차
식다
오라
가신
비정상
준서
신현석
규민
코리
유전체학
곽진
노특북
브레이
조선왕조
실록
제보
장윤
벗어나다
맥스
반짝거리다
랑은
스톤
교려
박희
수입
렌트
만여
아시냐
그래프
싸다
쓰앵님
대학교육
교우
키워지다
네요
객원
상윤
거슬리다
우상
대시
고형
미러링
나은
장정
뚫다
영미
단편
마이스페이스
결리다
황은진
웹서비스
적응하다
곽창모
박찬수
외부링크
라보
유용태
김유미
달전
민사재판
차갑다
블락
읺습니
굥유
야마모토
의뢰
조성
생생
장의
반시
정남
의아
이행
혼돈
내한
블럭
윤아
구입
끝내주다
잡아먹다
아뇨오전
화폐
융론
인강
그래픽스
육군사관학교
장현태
어쭤
뚫리다
임호
상대
김병휘
한컴
황로
황노학
메뉴탭
컬투
라가
이어트
전문가
슬슬
고프다
뷰잉
떳습니
김은정
독하다
이기성
현대인
짜장
장선
저작권법
영문학
정희
아보
계쏙
세이브
돌발상황
전문의
수만
한다해
별거
지희
파랑색
교과서
박진수
숙이다
신진
간걸
군용
불량
크레딧
어뗳
긋다
이염
무선랜
유선랜
독성학
번갈아
김건
기사
승락
리딩
정우
때로는
무기
무기명
이슙니
칼튜라
햘수
스위치
위대
거만
상호작용
초상권
공중보건학
알수
전선
인코딩
스케줄링
문성현
아지
율과
김연주
연두색
칠하다
만

In [67]:
minimum_count = 3
more_than_one_time = []
for i in tqdm(range(len(remove_stopwords))):
    tmp = remove_stopwords[i]
    if remove_stopwords.count(tmp) >= minimum_count:
        more_than_one_time.append(tmp)

100%|█████████████████████████████████████████████████████████████████████████| 144930/144930 [13:51<00:00, 174.24it/s]


In [69]:
Counter(remove_stopwords).most_common()

[('감사하다', 4561),
 ('수업', 2486),
 ('안녕하다', 2440),
 ('블랙보드', 2355),
 ('강의', 2354),
 ('학생', 2300),
 ('확인', 2274),
 ('교수', 1960),
 ('드리다', 1933),
 ('뜨다', 1784),
 ('출석', 1667),
 ('문제', 1397),
 ('문의', 1388),
 ('시험', 1348),
 ('방법', 1239),
 ('접속', 1036),
 ('영상', 1021),
 ('설정', 1006),
 ('들어가다', 978),
 ('조교', 950),
 ('제출', 933),
 ('과제', 902),
 ('녹화', 888),
 ('등록', 858),
 ('해결', 800),
 ('화면', 790),
 ('로그인', 781),
 ('코스', 754),
 ('해보다', 742),
 ('가능하다', 732),
 ('콜라보', 702),
 ('맞다', 697),
 ('락다운', 677),
 ('사용', 671),
 ('경우', 670),
 ('수강', 657),
 ('누르다', 651),
 ('파일', 649),
 ('과목', 631),
 ('올리다', 612),
 ('브라우저', 604),
 ('레이', 603),
 ('하나요', 590),
 ('자다', 568),
 ('께서', 552),
 ('진행', 548),
 ('답변', 547),
 ('보내다', 546),
 ('관련', 528),
 ('좋다', 522),
 ('자료', 522),
 ('해주다', 519),
 ('체크', 513),
 ('메뉴', 511),
 ('학기', 506),
 ('기록', 478),
 ('오류', 463),
 ('궁금하다', 446),
 ('업로드', 446),
 ('삭제', 440),
 ('해당', 437),
 ('메일', 434),
 ('연락', 432),
 ('링크', 423),
 ('교육', 420),
 ('건가', 407),
 ('여쭈다', 405),
 ('질문', 398),
 ('하